In [1]:
#| include: false

from src.imports import *

with open('src/get_ner_in_context.py') as f:
    exec(f.read())

# Preproceso

## Ingesta del documento

Leemos el libro El Quijote.

In [2]:
# Enviar una solicitud GET a la URL
response = requests.get("https://www.gutenberg.org/cache/epub/2000/pg2000.txt")

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    quijote = response.text
    # print("Éxito al leer el texto.")
else:
    print("Error al leer el texto.")

Nos ocupamos únicamente de los primeros seis capítulos de la primera parte de El Quijote.

In [3]:
### Inicio del primer capítulo

# Palabra objetivo
palabra_start = "Primera parte del ingenioso hidalgo don Quijote de la Mancha"

# Expresión regular para capturar todo lo que va antes de un patrón de  búsqueda
patron = re.compile(rf"{palabra_start}\s*(.*)", re.DOTALL)

coincidencia = patron.search(quijote)

if coincidencia:
    quijote = coincidencia.group(1) # 0 incluye la palabra del search, 1 no lo incluye
    console.print("[bold]Inicio del primer capítulo:\n\n[/bold]", quijote[:251])
    # print("Inicio del texto:\n\n", quijote[:251])
else:
    console.print("[bold]Inicio del texto:\n\n[/bold]", quijote[:251])
    

### Inicio del sépimo capítulo (y final del sexto)
palabra_end = "Capítulo VII"

# patron = re.compile(rf"(.*)\b{re.escape(palabra_end)}\b", re.DOTALL) # .* es codicioso ("greedy"): captura la mayor cantidad posible de caracteres antes de la coincidir con la palabra, captura hasta la última aparición de la palabra buscada
patron = re.compile(rf"(.*?)\b{re.escape(palabra_end)}\b", re.DOTALL) # .* es no codicioso ("lazy"): captura la menor cantidad posible de caracteres y evitar que el resto del texto cooincida, se detiene en la primera coincidencia

coincidencia = patron.search(quijote)

if coincidencia:
    quijote = coincidencia.group(0)
    console.print("[bold]Final del sexto capítulo:\n\n[/bold]", quijote[-231:])
    quijote = coincidencia.group(1) # Elimino el patrón de búsqueda
else:
    console.print("[bold]Final del texto:\n\n[/bold]", quijote[-231:])


Inicio del primer capítulo:

 Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua

Final del sexto capítulo:

 o el cura en oyendo el nombre— si tal libro hubiera
mandado quemar; porque su autor fue uno de los famosos poetas del mundo, no
sólo de España, y fue felicísimo en la tradución de algunas fábulas de
Ovidio.




Capítulo VII

## Carga del modelo

### spaCy

**[What’s spaCy?](https://spacy.io/usage/spacy-101#whats-spacy)** spaCy es una biblioteca gratuita de código abierto para el procesamiento avanzado del lenguaje natural en Python.

La librería spaCy se basa en modelos de aprendizaje automático que se entrenaron con grandes cantidades de datos de texto etiquetados.

Descarga y carga del modelo de lenguaje a través de la librería spaCy:

In [4]:
#| code-fold: false
# !python -m spacy download es_core_news_md

In [5]:
#| include: false
# import es_core_news_md
# nlp = es_core_news_md.load()

In [6]:
#| code-fold: false
import spacy
nlp = spacy.load("es_core_news_md")

## Fragmentación de la información

Podemos dividir el texto en párrafos.

In [7]:
#| layout-ncol: 3
for parrafo in list(nlp(quijote).sents)[:3]: # No es necesario pasarlo a una lista, solo para coger un número de ejemplos
    console.print("[blue]\nSiguiente párrafo:\n\n[/blue]", parrafo)
    #print(parrafo)

Siguiente párrafo:

 Capítulo primero.

Siguiente párrafo:

 Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor.

Siguiente párrafo:

 Una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda.

Y los párrafos en palabras.

In [8]:
for parrafo in list(nlp(quijote).sents)[1:2]:
    console.print("[blue]\nSiguiente párrafo:\n[/blue]")
    # print(colored("\nSiguiente párrafo:\n",'blue'))
    for palabra in parrafo[:5]:
        print(palabra)

Siguiente párrafo:

Que
trata
de
la
condición


Y separar cada una de las frases.

In [9]:
documento = nlp(quijote)
sents = list(documento.sents)[:3]
sents

[Capítulo primero.,
 Que trata de la condición y ejercicio del famoso hidalgo
 don Quijote de la Mancha
 
 En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
 tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
 rocín flaco y galgo corredor.,
 Una olla de algo más vaca que carnero,
 salpicón las más noches, duelos y quebrantos los sábados, lantejas los
 viernes, algún palomino de añadidura los domingos, consumían las tres
 partes de su hacienda.]

# Pipeline NPL

In [10]:
#| code-fold: false
text = quijote

## Tokenization

In [11]:
tokens = word_tokenize(text)
print(tokens[:10])

['Capítulo', 'primero', '.', 'Que', 'trata', 'de', 'la', 'condición', 'y', 'ejercicio']


## Stop Words

In [12]:
stop_words = set(stopwords.words('spanish'))
tokens = [w for w in tokens if w.lower() not in stop_words]
tokens_ejemplos = tokens
print(tokens[:10])

['Capítulo', 'primero', '.', 'trata', 'condición', 'ejercicio', 'famoso', 'hidalgo', 'don', 'Quijote']


## Stemming and Lemmatization

https://spacy.io/models/es  
https://github.com/explosion/spacy-models/releases?q=es_core_web_md&expanded=true

In [13]:
stemmer = SnowballStemmer("spanish")
tokens = [stemmer.stem(palabra) for palabra in tokens]
tokens[:10]

['capitul',
 'primer',
 '.',
 'trat',
 'condicion',
 'ejercici',
 'famos',
 'hidalg',
 'don',
 'quijot']

## POS Tagging

Part of speech.

Etiqueta las palabras de un texto determinado según sus respectivos tipos de palabras, como sustantivos, adjetivos, adverbios y verbos.

El etiquetado POS es el proceso de identificar la categoría gramatical (como verbo, sustantivo, adjetivo, etc.) de cada palabra en una oración.

Las etiquetas que spaCy utiliza se basan en el trabajo realizado por Universal Dependencies, un repositorio común que se puede utilizar para entrenar modelos como spaCy. La página de Dependencias universales tiene información sobre [los corpus disponibles para cada idioma](https://universaldependencies.org/).

Las etiquetas (**atributos del token**) que spaCy crea para cada token podemos encontrarlas en <https://spacy.io/api/token#attributes>.

Merece la pena prestar especial atención al atributo *pos_*, que nos permitirá encontrar palabras según su categoría gramatical.

<a href="https://universaldependencies.org/u/pos/" target="_blank" style="position: relative; width: 100%; height: 100%; top: 0; left: 0; z-index: 2;"></a>
<iframe src="https://universaldependencies.org/u/pos/" width="1000px" height="320px" scrolling="yes" style="border: none;"></iframe>

<div style="text-align: center;">
<a href="https://universaldependencies.org/u/pos/" target="_blank" style="text-decoration: none; color: blue;">
    Universal POS tags
  </a>
</div>

<span style="text-decoration:underline">**Ejemplo.**</span>

In [14]:
print("Token:", nlp(text)[0])

Token: Capítulo


In [15]:
#| code-fold: false
propiedades = ['text', 'lang_', 'pos_', 'is_digit', 'is_lower', 'is_upper', 'is_sent_start', 'is_sent_end',
               'like_email', 'like_url', # Espero no encontrar ninguna de estas etiquetas en El Quijote
               'sentiment', 'sent']

In [16]:
#| collapse: true
ejemplo_01 = nlp(text)[0]

for attr in propiedades:
    print("obj.%s = %r" % (attr, getattr(ejemplo_01, attr)))

obj.text = 'Capítulo'
obj.lang_ = 'es'
obj.pos_ = 'PROPN'
obj.is_digit = False
obj.is_lower = False
obj.is_upper = False
obj.is_sent_start = True
obj.is_sent_end = False
obj.like_email = False
obj.like_url = False
obj.sentiment = 0.0
obj.sent = Capítulo primero.


<span style="text-decoration:underline">**Ejemplo.**</span>

In [17]:
ejemplo_02 = nlp(re.sub(r'[^\w\s]', '', text))[:15]

# Encabezado
print(f'{"text":<12} {"lemma_":<12}{"pos":<12}{"pos_":<12}{"tag_":<12}{"dep_":<12}{"shape_":<12}{"is_alpha":<12}{"is_stop":<12}')
print("="*12*9)

# Loop que imprime los datos en formato de tabla
for token in ejemplo_02:
    print(f'{token.text:<12} {token.lemma_:<11} {token.pos:<11} {token.pos_:<11} {token.tag_:<11} {token.dep_:<12}'
          f'{token.shape_:<11} {token.is_alpha:<11} {token.is_stop:<11}')


text         lemma_      pos         pos_        tag_        dep_        shape_      is_alpha    is_stop     
Capítulo     Capítulo    96          PROPN       PROPN       nsubj       Xxxxx       1           0          
primero      primero     84          ADJ         ADJ         advmod      xxxx        1           1          
Que          Que         98          SCONJ       SCONJ       nsubj       Xxx         1           1          
trata        tratar      100         VERB        VERB        acl         xxxx        1           1          
de           de          85          ADP         ADP         case        xx          1           1          
la           el          90          DET         DET         det         xx          1           1          
condición    condición   92          NOUN        NOUN        obj         xxxx        1           0          
y            y           89          CCONJ       CCONJ       cc          x           1           1          
ejercicio    ejerc

### Selección de nombres

En nuestro ejemplo anterior:

In [18]:
nouns = []
for token in ejemplo_02:
    if token.pos_ == 'NOUN':
        nouns.append(token.lemma_)

nouns_total = Counter(nouns)

df = pd.DataFrame(nouns_total.most_common(), columns=['noun', 'count'])
df.style.hide(axis="index")

noun,count
condición,1
ejercicio,1
hidalgo,1
don,1


En nuestro texto completo:

In [19]:
nouns = []
for token in nlp(text):
    if token.pos_ == 'NOUN':
        nouns.append(token.lemma_)

nouns_total = Counter(nouns)

df = pd.DataFrame(nouns_total.most_common(), columns=['token', 'frecuencia'])
df[:5].style.hide(axis="index")

token,frecuencia
caballero,61
don,56
libro,48
señor,38
cura,33


### Selección de cualqueir categoría gramatical

Consultamos la Universal POS tags y definimos la categoría que queremos extraer.

In [20]:
#| code-fold: false
categoria = 'ADV'

In [21]:
lista_categoria = []
for token in nlp(text):
    if token.pos_ == categoria:
        lista_categoria.append(token.lemma_)

categoria_total = Counter(lista_categoria)

df = pd.DataFrame(categoria_total.most_common(), columns=['token', 'frecuencia'])
df[:5].style.hide(axis="index")

token,frecuencia
no,186
más,61
tanto,50
así,44
mucho,33


## NER

Named-Entity Recognition.

NER es el proceso de identificar entidades nombradas (como nombres de personas, lugares, organizaciones, etc.) en un texto. La biblioteca spaCy es particularmente útil para esta tarea.

Cada modelo de la librería spaCy tiene sus propias entidades, que se encuentran en la sección Label Scheme de la documentación del modelo. Para modelo cargado las entidades se almacenan con las etiqetas **LOC**, **ORG**, **PER** y **MISC**. (lugares, organizacoines, personas y un comodín para las tres anteriores)


In [22]:
#code-fold: false
getattr(nlp.get_pipe('ner'), 'labels')

('LOC', 'MISC', 'ORG', 'PER')


Todas las entidades nombradas se encuentran en la propiedad **document.ents**.

In [23]:
#| code-fold: false
documento = nlp(text)

In [24]:
for named_entity in list(documento.ents)[:10]:
    print(named_entity, named_entity.label_)

Que trata de la condición MISC
hidalgo PER
la Mancha LOC
Una olla de algo más vaca MISC
El resto della MISC
Tenía en su casa una MISC
Frisaba LOC
Quieren PER
Quijada PER
Quesada LOC


### Búsqueda de una entidad en el documento

Podemos encontrar una entidad dentro de su propio contexto.

iba aquí

In [25]:
get_ner_in_context('hidalgo', nlp(quijote), desired_ner_labels = False)

**Resultado 1.**

Tipo de entidad: **PER**

Contexto:

Que trata de la condición y ejercicio del famoso **hidalgo** don Quijote de la Mancha  En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un **hidalgo** de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor.

**Resultado 2.**

...

El labrador estaba admirado oyendo aquellos disparates; y, quitándole la visera, que ya estaba hecha pedazos de los palos, le limpió el rostro, que le tenía cubierto de polvo; y apenas le hubo limpiado, cuando le conoció y le dijo:  — Señor Quijana —que así se debía de llamar cuando él tenía juicio y no había pasado de **hidalgo** sosegado a caballero andante—, ¿quién ha puesto a vuestra merced desta suerte?  Pero él seguía con su romance a cuanto le preguntaba.

**Resultado 3.**

Tipo de entidad: **PER**

Contexto:

A esto respondió el labrador:  — Mire vuestra merced, señor, pecador de mí, que yo no soy don Rodrigo de Narváez, ni el marqués de Mantua, sino Pedro Alonso, su vecino; ni vuestra merced es Valdovinos, ni Abindarráez, sino el honrado **hidalgo del señor** Quijana.  

### Visualizers

[https://spacy.io/usage/visualizers](https://spacy.io/usage/visualizers)

Podemos usar sus demos online que actualmente usan la versión small del modelo en español.

- [https://demos.explosion.ai/displacy](https://demos.explosion.ai/displacy?text=Cuando%20Mario%20empez%C3%B3%20a%20trabajar%20en%20este%20pipeline%2C%20poca%20gente%20imaginaba%20el%20alcance.&model=es_core_news_sm&cpu=1&cph=1)
- [https://demos.explosion.ai/displacy-ent](https://demos.explosion.ai/displacy-ent?text=Cuando%20Mario%20empez%C3%B3%20a%20trabajar%20en%20este%20pipeline%2C%20poca%20gente%20imaginaba%20el%20alcance.&model=es_core_news_sm&ents=person%2Cnorp%2Cfacility%2Corg%2Cgpe%2Cloc%2Cproduct%2Cevent%2Cwork_of_art%2Clanguage%2Cdate%2Ctime%2Cpercent%2Cmoney%2Cquantity%2Cordinal%2Ccardinal%2Cmisc%2Cdrv%2Cevt%2Cgpe_loc%2Cgpe_org%2Cprod%2Cper)

#### Dependency parse

Análisis sintáctico de dependencia. Es el visualizador por defecto si no especificamos ningún estilo.

Visualizamos el texto ```Cuando Mario empezó a trabajar en este pipeline, poca gente imaginaba el alcance.``` como ejemplo.

In [26]:
#| echo: false
text_for_visual = "Cuando Mario empezó a trabajar en este pipeline, poca gente imaginaba el alcance."

In [27]:
#| code-fold: false
#| width: 50 %

# options: https://spacy.io/api/top-level#options-dep
options = {"compact": True,
           "color": "green",
           "bg:": "red", # no hace caso
           "arrow_width": 5,
           "word_spacing": 20,
           "distance": 120}

doc = nlp(text_for_visual)
displacy.render(doc, options=options) # = displacy.render(doc, style="dep")

In [28]:
#| include: false
print("\n")

#### Entity recognizer

Visualizando el reconocedor de entidades.

Usamos dos modelos distintos. Cada uno cuenta con sus propias entidades identificadas en el Label Scheme de cada modelo.

Visualizamos el texto ```María se fue en 2020 al río Guadiana a pescar peces coloridos mientras Mario iban a por bebida al supermercado Mercado.``` como ejemplo.

- El primer modelo, [es_core_news_md](https://spacy.io/models/es#es_core_news_md), el que hemos estado usando hasta ahora. Con cuatro entidades NER: LOC, MISC, ORG y PER.

In [29]:
#| echo: false
text_for_visual = "María se fue en 2020 al río Guadiana a pescar peces coloridos mientras Mario iban a por bebida al supermercado Mercado."

In [30]:
#| code-fold: false
#| width: 50 %

# options: https://spacy.io/api/top-level#displacy_options-ent

colors = {"PER": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
          "LOC": "linear-gradient(90deg, orange, lightblue)"}
options = {#"ents": ["PER"], # si quisiéramos especificar las entidades que queremos mostrar
           "colors": colors}

doc = nlp(text_for_visual)
#displacy.server(doc, style="ent", options=options, auto_select_port=True)
displacy.render(doc, style="ent", options=options)

- El segundo modelo, [en_core_web_sm](https://spacy.io/models/en#en_core_web_sm), en inglés. Con muchas más entidades con NER: CARDINAL, DATE, ..., WORK_OF_ART

In [31]:
#| code-fold: false
#| width: 50 %

# options: https://spacy.io/api/top-level#displacy_options-ent

colors = {"PERSON": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
          "DATE": "linear-gradient(90deg, white, green)",
          "GPE": "linear-gradient(90deg, orange, lightblue)"}
options = {#"ents": ["PER"], # si quisiéramos especificar las entidades que queremos mostrar
           "colors": colors}

nlp_en = spacy.load("en_core_web_sm")
doc = nlp_en(text_for_visual)
displacy.render(doc, style="ent", options=options)

### Selección de un tipo de entidad

Podemos identificar el tipo de entidades que queremos buscar.

<span style="text-decoration:underline">**Ejemplo.**</span>

LOC

In [32]:
print(f"{nlp(text).ents[2]} - {repr(nlp(text).ents[2].label_)}")

la Mancha - 'LOC'


<span style="text-decoration:underline">**Ejemplo.**</span>

PER

In [33]:
pers = []
for named_entity in nlp(text).ents:
    if named_entity.label_  == 'PER':
        pers.append(named_entity.lemma_)

pers_total = Counter(pers)

df = pd.DataFrame(pers_total.most_common(), columns=['named_entity', 'frecuencia'])
df[:6]

,named_entity,frecuencia
0,Andrés,7
1,Nicolás,5
2,señor caballero,5
3,Amadís de Gaula,4
4,Amadís,4
5,Rocinante,4


### Diferencia entre POS y NER

Podemos ver la diferencia entre POS y NER con el siguiente ejemplo.

<span style="text-decoration:underline">**Ejemplo.**</span>

```María se fue en 2020 al río Guadiana a pescar peces coloridos mientras Mario iban a por bebida al supermercado Mercado.```

In [34]:
#| echo: false
text_for_POS_NER = "María se fue en 2020 al río Guadiana a pescar peces coloridos mientras Mario iban a por bebida al supermercado Mercado."

ejemplo = nlp(text_for_POS_NER)

In [35]:
#| eval: false

propiedades = ['text', 'pos_']

print('POS')
for token in ejemplo[:6]:
    print('---')
    for attr in propiedades:
        print("obj.%s = %r" % (attr, getattr(token, attr)))


print('NER')
print('---')
for named_entity in list(ejemplo.ents):
   print(f"{named_entity} - label: {repr(named_entity.label_)}")

In [36]:
#| echo: false

from IPython.display import display, HTML

ejemplo = nlp(text_for_POS_NER)

# Primer chunk
output1 = []

propiedades = ['text', 'pos_']

output1.append('POS')
for token in ejemplo[:6]:
    output1.append('---')
    for attr in propiedades:
        output1.append(f"obj.{attr} = {repr(getattr(token, attr))}")
output1_html = "<br>".join(output1)  # Convertir a HTML con saltos de línea

# Segundo chunk
output2 = []

output2.append('NER')
output2.append('---')
for named_entity in list(ejemplo.ents):
   output2.append(f"{named_entity} - label: {repr(named_entity.label_)}")
output2_html = "<br>".join(output2)  # Convertir a HTML con saltos de línea


display(HTML(f"""
<div style="display: flex; justify-content: space-between;">
    <div style="margin-right: 20px; flex: 1;">
        <pre>{output1_html}</pre>
    </div>
    <div style="flex: 1;">
        <pre>{output2_html}</pre>
    </div>
</div>
"""))